<a href="https://colab.research.google.com/github/matheus97eng/desafio_solvimm/blob/main/notebook_de_apresentacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importando bibliotecas

In [2]:
import pandas as pd
import string, re, nltk
#from nltk import word_tokenize
from nltk.corpus import stopwords

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

nltk.download('stopwords')      # obtem as stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

- pandas - manipulação de dataframes
- nltk.word_tokenize - 
- nltk.corpus.stopwords - lista de palavras consideradas "stopwords"
- string - manipulação de strings
- re - biblioteca python para operação de strings com codificação. No caso iremos trabalhar com a codificação dos emojis.
- da biblioteca sklearn:
    - MultinomialMB - algoritmo de Naive Beyes do tipo multinominal, o mais indicado para multiclassificação
    - train_test_split - divide os dados em treino e teste
    - accuracy_score - método utilizado para validar o modelo

## obtenção dos dados e tratamento

Os dados foram fornecidos pela empresa SOLVIMM, que propôs o desafio. Eles correspondem a uma base de dados de produtos cadastrados pela empresa "Ponto Quente".

In [3]:
arq = 'https://github.com/matheus97eng/desafio_solvimm/blob/main/data/reviews.tsv?raw=true' # repositório do github
df_original = pd.read_csv(arq, sep='\t')
print(df_original.shape)
df_original.head()

(170583, 16)


,Unnamed: 0,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,product_category
0,762868,UK,29723892,R3VNENIATVV8QE,B00NOPQU2K,627793267,The Girl on the Train,5,0,1,N,N,Gripping you right where it matters,I know to say a story is &#34;gripping&#34; is...,2015-04-27,Digital_Ebook_Purchase
1,1284183,UK,41072087,R2U3LV67N99770,B0013F2LSK,13214624,11,5,2,5,N,Y,The Best of Me,"This album is totally fantastic, a great mix o...",2008-03-18,Music
2,1599315,UK,49938094,R3RO94POCHNI9V,B005CVWWJY,769273676,Ready Player One,5,0,0,N,Y,superb,Enjoyed every second of this book. It took me...,2014-08-28,Digital_Ebook_Purchase
3,204782,UK,14398213,R3S2BB5SBWBC1,B00008AWV3,841759677,The Four Feathers [DVD] [1939],5,1,1,N,Y,"Sweeping, authentic historic drama",I loved the historic scenes---the English coun...,2013-12-27,Video DVD
4,352938,UK,20140500,R27E2PNXJSWJIN,B00FAXJHCY,803172158,The Martian,4,0,0,N,Y,... a few pages to get through it but a good b...,May have skipped a few pages to get through it...,2015-06-06,Digital_Ebook_Purchase


### Informações das features, exclusão de dados nulos e explicação do modelo:

Vamos obter uma visão geral das features em questão.

In [4]:
display(df_original.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170583 entries, 0 to 170582
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Unnamed: 0         170583 non-null  int64 
 1   marketplace        170583 non-null  object
 2   customer_id        170583 non-null  int64 
 3   review_id          170583 non-null  object
 4   product_id         170583 non-null  object
 5   product_parent     170583 non-null  int64 
 6   product_title      170583 non-null  object
 7   star_rating        170583 non-null  int64 
 8   helpful_votes      170583 non-null  int64 
 9   total_votes        170583 non-null  int64 
 10  vine               170583 non-null  object
 11  verified_purchase  170583 non-null  object
 12  review_headline    170583 non-null  object
 13  review_body        170582 non-null  object
 14  review_date        170578 non-null  object
 15  product_category   170583 non-null  object
dtypes: int64(6), object(

None

O desafio nos pede para classificar os produtos somente de acordo com os reviews feitos pelos clientes. Esclarecido isso, não precisamos nos preocupar com outras features, a não ser: `review_headline`, que é o título da avaliação, `review_body`, que é a avaliação em si e por fim, `product_category`, que é a categorização corrigida do produto. Aqui não consideraremos relevante a data de postagem do review, nem o ID do review.

Dado que as features `review_headline` e `product_category` são todas texto (do tipo caracter), e por se tratar de um problema de classificação (identificar qual a classe que o produto pertence), precisamos de um modelo que utilize NPL (Natural Language Processing). Será escolhido o algoritmo de [Nave Beyes](https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/), pois é um algoritmo de fácil aplicação com python e sklearn, tem uma boa resposta dado uma quantidade pequena de dados e não é um algoritmo pesado. 

Uma desvantagem desse modelo é que ele não considera a semântica do texto. Por exemplo, ao olhar para a frase "meus dedos ficaram muito apertados quando eu testei na corrida". Analisando o contexto, poderíamos identificar que o produto se trata provavelmente de um tênis. Mas o Nave Beyes analisa palavra por palavra, separadamente, o que tornaria mais difícil a identificação, nesse exemplo dado.

Explicando de uma forma não formal, o algoritmo de Nave Beyes trabalha calculando qual é a probabilidade de um produto ser da classe "X" sendo que no review desse produto encontramos certas "palavras-chave". Treinando o modelo, ele consegue identificar, por exemplo que, se no review aparecer a palavra "bola", é mais provável que o produto seja da categoria "esportes". Com as informações da base de dados, o algoritmo estará treinado e validado para classificar outros produtos, fora da base de dados preparada pelo time de dados da "Ponto Quente". 

No entanto, antes de modelar nosso problema, precisamos tratar os nossos dados.
Executando `df_original.info()` já vemos que a feature `review_body` possui uma linha com dado nulo. Isso porque vemos 170582 dados não nulos nessa coluna, enquanto que a nossa base de dados possui 170583 linhas. Vamos optar por excluir toda a linha que contém esse dado nulo, já que atrapalhará no desenvolvimento do modelo e se trata apenas de um produto.

In [5]:
excluir = df_original[df_original['review_body'].isnull()].index
df_tratado = df_original.drop(excluir).reset_index()

### Tipos de classificação

É importante também olharmos para os tipos de classificação de produtos que a empresa tem. Na base fornecida, foram identificados 32 classes. É essencial entender que o modelo a ser treinado **não identificará classes de produtos que não estão na base de dados**. Desse modo, se a empresa quisesse identificar um produto como classe "carro" ou não, teria que acrescentar à base de dados vários produtos da categoria "carro".

O modelo de machine learning não consegue interpretar dados do tipo string. Portanto, precisamos alterar os dados da coluna alvo `product_category`, fazendo uma tokerização simples, em outras palavras, substituindo as palavras por números.

In [10]:
df_tratado['num_category'] = df_tratado['product_category'].map({'Digital_Ebook_Purchase':0, 
       'Music':1, 'Video DVD':2, 'Mobile_Apps':3, 'Books':4, 'Electronics':5, 
       'Toys':6, 'Video Games':7, 'Digital_Video_Download':8, 'Digital_Music_Purchase':9, 
       'PC':10, 'Camera':11, 'Baby':12, 'Wireless':13, 'Home Entertainment':14, 'Sports':15,
       'Musical Instruments':16, 'Lawn and Garden':17, 'Home Improvement':18, 'Home':19, 
       'Watches':20, 'Video':21, 'Shoes':22, 'Office Products':23, 'Automotive':24, 
       'Health & Personal Care':25, 'Personal_Care_Appliances':26, 'Software':27, 
       'Kitchen':28, 'Luggage':29, 'Pet Products':30, 'Beauty':31})

0         0
1         1
2         0
3         2
4         0
         ..
170577    4
170578    2
170579    1
170580    1
170581    3
Name: num_category, Length: 170582, dtype: int64

### Features de review dos clientes

Há duas colunas no dataframe com o conteúdo de avaliações dos clientes: `review_headline`, que contém o título da avaliação, e `revew_body`, que contém o corpo do review.

Inicialmente, vamos juntar todas as palavras dessas duas features em uma coluna apenas, já que o contexto do texto não importa no modelo de Naive Beyes. 

In [11]:
chars = [(df_tratado['review_headline'].iloc[i] + ' ' + df_tratado['review_body'].iloc[i])
         for i in df_tratado.index.to_list()]
df_tratado['review_total'] = chars

In [12]:
emoji_pattern = re.compile("["
        u"\U0001F300-\U0001F5FF"  # símbolos
        u"\U0001F680-\U0001F6FF"  # transporte e símbolos de mapa
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0000231A-\U000023F3"  # relógios e setas
        u"\U000026A1-\U000026BE"  # relâmpago, cores e bolas de esportes
        u"\U00002753-\U00002757"  # pontuação
        u"\U00002B50"             # estrela
        u"\U0001F32D-\U0001F37F"  # comidas
        u"\U0001F3A0-\U0001F3D3"  # esportes
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F910-\U0001F93E"  # mais emoticons e emojis de esportes
                           "]+", flags=re.UNICODE)
emoji_pattern

re.compile(r'[🌀-🗿🚀-\U0001f6ff\U0001f1e0-🇿⌚-⏳⚡-⚾❓-❗⭐🌭-🍿🎠-🏓😀-🙏🤐-🤾]+', re.UNICODE)

In [13]:
#def clean_text()

col_corrigida = []
char_excluir = string.punctuation + string.digits

for row in df_tratado.index.to_list():
    temp = [char for char in df_tratado['review_total'].iloc[row] if char not in char_excluir]
    text = ''.join(temp).lower()
    text = emoji_pattern.sub(r'', text)
    for word in stopwords.words('english'):
        text.replace(word, '')
    col_corrigida.append(text)

In [14]:
df_tratado['review_total'] = col_corrigida
df_tratado['review_total']

df_ML = df_tratado[['review_total', 'num_category']]

## preparação final para aplicar o algoritmo:

In [15]:
x_train, x_test, y_train, y_test = train_test_split(df_ML['review_total'], df_ML['num_category'], random_state=50)

vect = CountVectorizer(ngram_range=(2,2))
X_train = vect.fit_transform(x_train)
X_test = vect.transform(x_test)

mnb = MultinomialNB(alpha =0.2)

mnb.fit(X_train,y_train)

result= mnb.predict(X_test)
print(result)

[2 2 3 ... 2 4 1]


In [16]:
accuracy_score(result,y_test)

0.7324250808985603